# 0. Setup

https://wikidocs.net/book/14314

# 1 LangChain Tutorial

In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

## 1.1 Large Language Model

LLM Options:
* model_name(model): the name of the model
* temperature: 0 ~ 2, larger is more random
* max_tokens: depends on the model, the maximum number of tokens to generate
* base_url: the base URL of the API
* api_key: the API key

### 1.1.1 OpenAI

OpenAI Document: https://platform.openai.com/docs/overview

In [1]:
from langchain_openai import ChatOpenAI

model_openai = ChatOpenAI(
    model='gpt-4o-mini',  # latest model
    temperature=0.2,
    max_tokens=50
)

### 1.1.2 Ollama - Open Source LLM

Ollama local
* model options: `~/.ollama/models` (Mac) or `/usr/share/ollama/.ollama/models` (Linux)
* `gemma2:9b-instruct-q4_0`: 9 billion parameters with instruct tuning and 4-bit quantization
* modelfile: A model file is the blueprint to create and share models with Ollama.
     
    ```shell
    FROM $HOME/.ollama/models/blobs/sha256-aqoqkj13ik4ujn9oducnzkjdn
    TEMPLATE "<start_of_turn>user
    {{ if .System }}{{ .System }} {{ end }}{{ .Prompt }}<end_of_turn>
    <start_of_turn>model
    {{ .Response }}<end_of_turn>
    "
    PARAMETER stop <start_of_turn>
    PARAMETER stop <end_of_turn>
    ```
    
    * `PARAMETER`: 
        * temperature: 0 ~ 2, higher is more creative, lower is more coherent
        * num_ctx: context window size
    * `SYSTEM`: custom system message 
* Ollama CoLab
* Ollama LangChain: must install `langhain-ollama` seperately

In [3]:
import os

print(f"Ngrok URL: {os.getenv('NGROK_URL')}")
print(f"LangChain Tracing: {os.getenv('LANGCHAIN_TRACING_V2')}")
print(f"LangChain Project Name: {os.getenv('LANGCHAIN_PROJECT')}")

Ngrok URL: https://square-sunfish-vaguely.ngrok-free.app
LangChain Tracing: false
LangChain Project Name: sample_app


In [4]:
from langchain_ollama.chat_models import ChatOllama

model_ollama = ChatOllama(
    model='gemma:2b-instruct', 
    temperature=0, 
    base_url=os.getenv('NGROK_URL')
)

## 1.2 LECL: LangChain Expression Language 

LangChain use `Runnable` interface to chain together a series of operations. Any two runnables can be 'chained' together into sequences. We can use `.pipe` method or `|` operator to chain operations together. The output of the previous runnable's `.invoke()` call is passed as input to the next runnable. 

* `invoke`: call the chain of operations
* `stream`: stream the output of the chain of operations
* `batch`: call the chain of operations with batch inputs

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template('What is {topic}? Please answer in one sentence.')

chain_openai = (prompt | model_openai | StrOutputParser())
chain_pipe_ollama = (prompt.pipe(model_ollama).pipe(StrOutputParser()))

print('Class Type of `chain` object:', type(chain_openai))
print('# 1 `chain_openai`')
print(chain_openai.invoke(input={'topic': '1 + 1'}))
print('# 2 `chain_pipe_ollama`')
print(chain_pipe_ollama.invoke(input={'topic': '1 + 1'}))

Class Type of `chain` object: <class 'langchain_core.runnables.base.RunnableSequence'>
# 1 `chain_openai`
1 + 1 equals 2.
# 2 `chain_pipe_ollama`
1 + 1 = 2.


Useful `Runnable` classes:

* `RunnablePassthrough` is a special runnable that passes the input to the output without any modification.
    * if call `.assign` method, the output of the chain of operations is assigned to the variable.
* `RunnableLambda` can be used to run a user-defined function on the input.
* `RunnableParallel` is useful for running multiple runnables in parallel, but can also be useful for manipulating the output of one `Runnable` to match the input format of the next `Runnable` in a sequence.

In [6]:
from langchain.globals import set_verbose, set_debug
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

set_debug(True)
set_verbose(False)
prompt = PromptTemplate.from_template('What is two words that start with {letter}?')

chain = (
    RunnablePassthrough()
    | prompt
)
print('RunnablePassthrough:\n  ', chain.invoke(input='A'))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "A"
}
[chain/start] [chain:RunnableSequence > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "A"
}
[chain/end] [chain:RunnableSequence > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "A"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "input": "A"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[chain/end] [chain:RunnableSequence] s] Exiting Chain run with output:
[outputs]
RunnablePassthrough:
   text='What is two words that start with A?'


In [7]:
chain = (
    RunnablePassthrough().assign(letter=lambda x: x['letter'].upper())
    | prompt
)
print('RunnablePassthrough with assign:\n  ', chain.invoke(input={'letter': 'a'}))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "letter": "a"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<letter>] Entering Chain run with input:
{
  "letter": "a"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<letter> > chain:RunnableParallel<letter>] Entering Chain run with input:
{
  "letter": "a"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<letter> > chain:RunnableParallel<letter> > chain:RunnableLambda] Entering Chain run with input:
{
  "letter": "a"
}
[chain/end] [chain:RunnableSequence > chain:RunnableAssign<letter> > chain:RunnableParallel<letter> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "A"
}
[chain/end] [chain:RunnableSequence > chain:RunnableAssign<letter> > chain:RunnableParallel<letter>] s] Exiting Chain run with output:
{
  "letter": "A"
}
[chain/end] [chain:RunnableSequence > chain:RunnableAssign<letter>] s] Exiting Chain run with output:
{
  "letter": "A"
}
[ch

In [8]:
chain = (
    RunnableLambda(lambda x: x['letter'].upper()) 
    | prompt
)
print('RunnableLambda:\n  ', chain.invoke(input={'letter': 'a'}))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "letter": "a"
}
[chain/start] [chain:RunnableSequence > chain:RunnableLambda] Entering Chain run with input:
{
  "letter": "a"
}
[chain/end] [chain:RunnableSequence > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "A"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "input": "A"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[chain/end] [chain:RunnableSequence] s] Exiting Chain run with output:
[outputs]
RunnableLambda:
   text='What is two words that start with A?'


In [9]:
set_debug(False)
set_verbose(False)
chain.get_graph().print_ascii()

    +-------------+      
    | LambdaInput |      
    +-------------+      
            *            
            *            
            *            
       +--------+        
       | Lambda |        
       +--------+        
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
+----------------------+ 
| PromptTemplateOutput | 
+----------------------+ 


In [10]:
import random
chain = RunnableParallel(
    passed = RunnablePassthrough(),
    extra = RunnableLambda(lambda x: x['letter'].upper()),
    modefied = lambda x: random.choice(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'))
)
chain.invoke(input={'letter': 'a'})

{'passed': {'letter': 'a'}, 'extra': 'A', 'modefied': 'K'}

In [11]:
from pprint import pprint
chain1 = (
    RunnablePassthrough()
    | PromptTemplate.from_template('What is two words that start with {letter}?')
    | model_openai
    | StrOutputParser()
)
chain2 = (
    RunnablePassthrough()
    | PromptTemplate.from_template('What is two words that ends with {letter}?')
    | model_openai
    | StrOutputParser()
)
chain = RunnableParallel(
    start=chain1,
    end=chain2
)
output = chain.invoke(input={'letter': 'a'})
print('Output:\n')
pprint(output)
print('Graph:')
chain.get_graph().print_ascii()

Output:

{'end': 'Here are two words that end with the letter "a": \n'
        '\n'
        '1. Arena\n'
        '2. Panda',
 'start': 'Here are two words that start with the letter "A": \n'
          '\n'
          '1. Apple\n'
          '2. Adventure'}
Graph:
            +--------------------------+             
            | Parallel<start,end>Input |             
            +--------------------------+             
                 **               **                 
              ***                   ***              
            **                         **            
  +-------------+                  +-------------+   
  | Passthrough |                  | Passthrough |   
  +-------------+                  +-------------+   
          *                               *          
          *                               *          
          *                               *          
+----------------+                +----------------+ 
| PromptTemplate |                | 

### Quiz

```python
# travel information searching
inputs = {'country': 'Korea', 'city': 'Jeonju'}
# chain 1: find the two famous restaurant in the city
# chain 2: find the three famous tourist spot in the city
# chain 3: summarize the information in one paragraph chain 1 and chain 2
```

Total Graph

```plaintext
              +----------------------+               
              | Parallel<c1,c2>Input |               
              +----------------------+               
                 **               **                 
            **                         **            
  +-------------+                  +-------------+   
  | Passthrough |                  | Passthrough |   
  +-------------+                  +-------------+   
          *                               *          
+----------------+                +----------------+ 
| PromptTemplate |                | PromptTemplate | 
+----------------+                +----------------+ 
          *                               *          
  +------------+                    +------------+   
  | ChatOllama |                    | ChatOllama |   
  +------------+                    +------------+   
          *                               *          
+-----------------+              +-----------------+ 
| StrOutputParser |              | StrOutputParser | 
+-----------------+              +-----------------+ 
                 **               **                 
                      **     **                      
              +-----------------------+              
              | Parallel<c1,c2>Output |              
              +-----------------------+              
                          *                          
                 +----------------+                  
                 | PromptTemplate |                  
                 +----------------+                  
                          *                          
                   +------------+                    
                   | ChatOllama |                    
                   +------------+                    
                          *                          
                +-----------------+                  
                | StrOutputParser |                  
                +-----------------+                  
                          *                          
              +-----------------------+              
              | StrOutputParserOutput |              
              +-----------------------+    
```

In [ ]:
from pprint import pprint
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

os.environ['LANGCHAIN_TRACING_V2'] = 'true'  # use 'false' to disable tracing if not needed

model_openai = ChatOpenAI(
    model='gpt-4o-mini',  # latest model
    temperature=0.2,
    max_tokens=500
)

model_ollama = ChatOllama(
    model='gemma2:9b', 
    temperature=0, 
    base_url=os.getenv('NGROK_URL')
)

model = model_ollama
chain1 = (
    # a1: RunnablePassthrough or itemgetter
    # a2: PromptTemplate.from_template(  ?  )
    # | model
    # | StrOutputParser()
)
chain2 = (
    # b1: RunnablePassthrough or itemgetter
    # b2: PromptTemplate.from_template(  ?  )
    # | model
    # | StrOutputParser()
)
chain = (
    # RunnableParallel(c1=chain1, c2=chain2) 
    # c1: PromptTemplate.from_template(  ?  )
    # | model
    # | StrOutputParser()
)
output = chain.invoke(input= {'country': 'Korea', 'city': 'Jeonju'})
print('Output:\n')
print(output)
print('Graph:')
os.environ['LANGCHAIN_TRACING_V2'] = 'false'
chain.get_graph().print_ascii()

## 1.3 Prompt

A **prompt** is natural language text describing the task that an AI should perform. It can contain information like the instruction or question passing to the model and include other details such as context, inputs, or examples. We can use these elements to instruct the model better and get better results.

### 1.3.1 Prompt Template

`PromptTemplate` helps to translate user input and parameters into instructions for a language model.
* input: a dictionary of parameters
* output: `PromptValue`

In [12]:
# PromptTemplate
from datetime import datetime as dt
from langchain_core.prompts import PromptTemplate

template = 'What is {country} capital city?'

prompt1 = PromptTemplate.from_template(template)
prompt2 = PromptTemplate(
    template=template, 
    input_variables=['country']
)
prompt3 = PromptTemplate(
    template='Date: {today}\n' + template, 
    input_variables=['country'], 
    partial_variables={'today': dt.now().strftime('%Y-%m-%d')}  # predefined partial values
)

print('prompt1:', prompt1.invoke({'country': 'Korea'}))
print('prompt2:', prompt2.invoke({'country': 'Korea'}))
print('prompt3:', prompt3.invoke({'country': 'Korea'}))
try:
    prompt2.invoke({'c': 'Korea'})
except Exception as e:
    print(e)

prompt1: text='What is Korea capital city?'
prompt2: text='What is Korea capital city?'
prompt3: text='Date: 2024-08-05\nWhat is Korea capital city?'
"Input to PromptTemplate is missing variables {'country'}.  Expected: ['country'] Received: ['c']"


`ChatPromptTemplate` is used to format a list of messages.

In [13]:
# ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt1 = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant'),
    ('user', 'Tell me a joke about {topic}')
])
prompt2 = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant'),
    MessagesPlaceholder('msgs')
])

print('prompt1')
print(prompt1.invoke({'topic': 'cats'}))
print()
print('prompt2')
print(prompt2.invoke({'msgs': [HumanMessage(content='hi!')]}))

prompt1
messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='Tell me a joke about cats')]

prompt2
messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='hi!')]


### 1.3.2 Few shot prompts

One common prompting technique for achieving better performance is to include examples as part of the prompt. 

In [14]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

model_ollama = ChatOllama(
    model='gemma2:9b', 
    temperature=0, 
    base_url=os.getenv('NGROK_URL')
)
# translation: English to Korean
examples = [
    {'input': 'Hello', 'output': '안녕'},
    {'input': 'Goodbye', 'output': '잘가'},
    {'input': 'Thank you', 'output': '고마워'}
]
instruction = 'Translate english to Korean.'
example_template = 'Input: {input} > Output: {output}'

# zero-shot prompting
prompt1 = PromptTemplate(
    template=instruction+'\n'+example_template, 
    input_variables=['input'], 
    partial_variables={'output': ''}
)
chain1 = (
    prompt1
    | model_ollama
    | StrOutputParser()
)
# few-shot prompting
prompt2 = FewShotPromptTemplate(
    prefix=instruction,
    examples=examples,
    example_prompt=PromptTemplate.from_template(template=example_template),
    suffix='Input: {input} > Output:',
    input_variables=['input'],
)
chain2 = (
    prompt2
    | model_ollama
    | StrOutputParser()
)
print('# Chain1\n[Prompt]')
print(prompt1.invoke(input='deep learning').text)
print('[Output]')
print(chain1.invoke(input='deep learning'))
print('===='*10)
print('# Chain2\n[Prompt]')
print(prompt2.invoke(input='deep learning').text)
print('[Output]')
print(chain2.invoke(input='deep learning'))

# Chain1
[Prompt]
Translate english to Korean.
Input: deep learning > Output: 
[Output]
깊은 학습  (kireun haksaep) 


Let me know if you have any other phrases you'd like translated! 😊 

# Chain2
[Prompt]
Translate english to Korean.

Input: Hello > Output: 안녕

Input: Goodbye > Output: 잘가

Input: Thank you > Output: 고마워

Input: deep learning > Output:
[Output]
Input: deep learning > Output: 딥러닝 


Let me know if you'd like to translate anything else! 😊  



## 1.4 Output Parser

Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

* Get format instructions: A method which returns a string containing instructions for how the output of a language model should be formatted.
* Parse: A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

In [15]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser, JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama.chat_models import ChatOllama

model_ollama = ChatOllama(
    model='gemma2:9b', 
    temperature=0, 
    base_url=os.getenv('NGROK_URL'),
    num_ctx=500
)

model_openai = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0,
    max_tokens=500
)

In [16]:
# Comma Sprerated List
output_parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template='List the five city of {country}.\n{format_instructions}',
    input_variables=['country'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)
chain1 = prompt | model_ollama | output_parser
chain2 = prompt | model_openai | output_parser

print('[Prompt]')
print(prompt.invoke(input={'country': 'Korea'}).text)
print('[Output: Gemma2]')
print(chain1.invoke(input={'country': 'Korea'}))  # gemma2 is too nice
print('[Output: OpenAI]')
print(chain2.invoke(input={'country': 'Korea'}))

[Prompt]
List the five city of Korea.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
[Output: Gemma2]
['Seoul', 'Busan', 'Daegu', 'Incheon', "Gwangju \n\n\nLet me know if you'd like to know more about these cities! 😊"]
[Output: OpenAI]
['Seoul', 'Busan', 'Incheon', 'Daegu', 'Gwangju']


In [17]:
# Json 
# - Pydantic: data validation library for Python
from pprint import pprint
from langchain.pydantic_v1 import BaseModel, Field

class City(BaseModel):
    name: str = Field(description='City name')
    population: int = Field(description='City population')

class CityList(BaseModel):
    cities: list[City] = Field(description='List of cities ')

output_parser = JsonOutputParser(pydantic_object=CityList)

prompt = PromptTemplate(
    template='List two cities in {country} and tell me the population of these two cities.\n{format_instructions}',
    input_variables=['country'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)
chain1 = prompt | model_ollama | output_parser
chain2 = prompt | model_openai | output_parser

print('[Prompt]')
pprint(prompt.invoke(input={'country': 'Korea'}).text)
print()
print('[Output: Gemma2]')
print(chain1.invoke(input={'country': 'Korea'}))
print('[Output: OpenAI]')
print(chain2.invoke(input={'country': 'Korea'}))

[Prompt]
('List two cities in Korea and tell me the population of these two cities.\n'
 'The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"cities": {"title": "Cities", "description": "List of cities '
 '", "type": "array", "items": {"$ref": "#/definitions/City"}}}, "required": '
 '["cities"], "definitions": {"City": {"title": "City", "type": "object", '
 '"properties": {"name": {"title": "Name", "description": "City name", "type": '
 '"string"}, "population": {"title": "Population", "description": "City '
 'population", "

`.with_structured_output()` method is implemented for models that provide native APIs for structuring outputs, like tool/function calling or JSON mode, and makes use of these capabilities under the hood.

`WARNINGS`: only support for OpenAI for now.
* support list: [https://python.langchain.com/v0.2/docs/integrations/chat/](https://python.langchain.com/v0.2/docs/integrations/chat/)

In [18]:
# output pydantic object
class City(BaseModel):
    name: str = Field(description='City name')
    population: int = Field(description='City population')

class CityList(BaseModel):
    cities: list[City] = Field(description='List of cities ')

llm = model_openai.with_structured_output(CityList)
chain = prompt | llm
output = chain.invoke(input={'country': 'Korea'})

print(f'[Pydantic Object]: {type(output)}')
print(output)

# output Json object
from typing_extensions import Annotated, TypedDict
class City2(TypedDict):
    name: Annotated[str, ..., 'City name']
    population: Annotated[int, ..., 'City population']

class CityList2(TypedDict):
    cities: Annotated[list[City2], 'List of cities']

llm = model_openai.with_structured_output(CityList2)
chain = prompt | llm
output = chain.invoke(input={'country': 'Korea'})

print(f'[Json Object]: {type(output)}')
print(output)

[Pydantic Object]: <class '__main__.CityList'>
cities=[City(name='Seoul', population=9776000), City(name='Busan', population=3406000)]
[Json Object]: <class 'dict'>
{'cities': [{'name': 'Seoul', 'population': 9776000}, {'name': 'Busan', 'population': 3406000}]}


In [19]:
# model oriented format output
model_ollama = ChatOllama(
    model='gemma2:9b',
    temperature=0, 
    base_url=os.getenv('NGROK_URL'),
    num_ctx=500,
    format='json'
)

model_openai = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0,
    max_tokens=500,
    model_kwargs = {'response_format': {'type': 'json_object'}}
)

prompt = PromptTemplate(
    template='''List two cities in {country} and tell me the population of these two cities.
Return the answer as a list of JSON object. e.g., [{{"key1": "value1", "key2": "value2"}}, ...] 
Each JSON object should have keys and their values: 
| "name" | "type": "string", "description": "the name of the city"
| "population" | "type": "integer", "description": "the population of the city"
''',
    input_variables=['country'],
    partial_variables={'schema': CityList.schema()}
)
chain1 = prompt | model_ollama
chain2 = prompt | model_openai
o1 = chain1.invoke(input={'country': 'Korea'})
o2 = chain2.invoke(input={'country': 'Korea'})
# the output of model is AI
print(f'[Output: Gemma2] {type(o1.content)}')
print(o1.content)
print(f'[Output: OpenAI] {type(o2.content)}')
print(o2.content)

[Output: Gemma2] <class 'str'>
{
  "cities": [
    {
      "name": "Seoul",
      "population": 9605000
    },
    {
      "name": "Busan",
      "population": 3400000
    }
  ]
} 



[Output: OpenAI] <class 'str'>

{
  "cities": [
    {
      "name": "Seoul",
      "population": 9776000
    },
    {
      "name": "Busan",
      "population": 3406000
    }
  ]
}


## 1.5 Chain of Thoughts Prompting

Chain of thoughts prompting is a series of intermediate reasoning steps that guide the model to the final answer.

`GSM8K (Grade School Math 8K)` is a dataset of 8.5K high quality linguistically diverse grade school math word problems. The dataset was created to support the task of question answering on basic mathematical problems that require multi-step reasoning.

In [20]:
import pandas as pd

def process_data(df: pd.DataFrame) -> list[dict[str, str]]:
    df.rename(columns={'answer': 'cot_answer'}, inplace=True)
    df['cot'] = df['cot_answer'].apply(lambda x: x.split('####')[0].strip())
    df['answer'] = df['cot_answer'].apply(lambda x: x.split('####')[1].strip())
    return df.to_dict(orient='records')

df = pd.read_csv('GSM8K_test.csv')
examples = df.loc[range(0, 5)].copy()
queries = df.loc[range(5, 8)].copy()
examples = process_data(examples)
queries = process_data(queries)

print(f'[Question]')
for x in examples[0]["question"].split(". "):
    print(x, end='.\n')
print(f'[Chain of Thought]\n{examples[0]["cot"]}')
print(f'[Answer]\n{examples[0]["answer"]}')

[Question]
Janet’s ducks lay 16 eggs per day.
She eats three for breakfast every morning and bakes muffins for her friends every day with four.
She sells the remainder at the farmers' market daily for $2 per fresh duck egg.
How much in dollars does she make every day at the farmers' market?.
[Chain of Thought]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
[Answer]
18


In [34]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.pydantic_v1 import BaseModel, Field

class Answer(BaseModel):
    final_answer: str = Field(description='A single numbers for final answer to the question')

output_parser = JsonOutputParser(pydantic_object=Answer)

model_ollama = ChatOllama(
    model='gemma2:9b', 
    temperature=0.3, 
    base_url=os.getenv('NGROK_URL'),
    format='json'
)

In [47]:
# Baseline: zero-shot prompt w/o chain of thoughts
system = '''You are an excellent math solver to help user. 
Please output only the final answer without any explanation. 
The format should be {{'final_answer': 'answer'}}.
Please answer the following question. 
'''

zero_shot_template = ChatPromptTemplate.from_messages(
    [
        ('system', system),
        ('human', 'Question: {question}'),
    ]
)

chain = (
    RunnablePassthrough()
    | model_ollama
    | output_parser
)

output = chain.batch(inputs=[
    zero_shot_template.format_messages(question=q['question']) for q in queries
])

print('[zero-shot prompt w/o CoT]')
for i, o in enumerate(output):
    print('[Question]')
    pprint(f'{queries[i]["question"]}', width=120)
    print(f'[Answer] {queries[i]["answer"]}')
    print(f'[Predicted] {o["final_answer"]}')
    print()

[zero-shot prompt w/o CoT]
[Question]
('Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only '
 '60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?')
[Answer] 64
[Predicted] 72

[Question]
('Toulouse has twice as many sheep as Charleston. Charleston has 4 times as many sheep as Seattle. How many sheep do '
 'Toulouse, Charleston, and Seattle have together if Seattle has 20 sheep?')
[Answer] 260
[Predicted] 160

[Question]
('Carla is downloading a 200 GB file. Normally she can download 2 GB/minute, but 40% of the way through the download, '
 'Windows forces a restart to install updates, which takes 20 minutes. Then Carla has to restart the download from the '
 'beginning. How load does it take to download the file?')
[Answer] 160
[Predicted] 100



In [48]:
# few-shot prompt w/o chain of thoughts
system = '''You are an excellent math solver to help user. 
Please output only the final answer without any explanation. 
The format should be {{'final_answer': 'answer'}}.
Please answer the following question. 
'''

example_template = ChatPromptTemplate.from_messages(
    [
        ('human', 'Question: {question}'),
        ('ai', '{{"final_answer": {answer}}}'),
    ]
)
keys = ['question', 'answer']
few_shot_examples = FewShotChatMessagePromptTemplate(
    examples=[{k: e[k] for k in keys} for e in examples],   # filter with keys
    example_prompt=example_template
)
few_shot_template = ChatPromptTemplate.from_messages(
    [
        ('system', system),
        few_shot_examples,
        ('human', 'Question: {question}'),
    ]
)

chain = (
    RunnablePassthrough()
    | model_ollama
    | output_parser
)

output = chain.batch(inputs=[
    few_shot_template.format_messages(question=q['question']) for q in queries
])

print('[few-shot prompt w/o CoT]')
for i, o in enumerate(output):
    print('[Question]')
    pprint(f'{queries[i]["question"]}', width=120)
    print(f'[Answer] {queries[i]["answer"]}')
    print(f'[Predicted] {o["final_answer"]}')
    print()

[few-shot prompt w/o CoT]
[Question]
('Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only '
 '60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?')
[Answer] 64
[Predicted] 72

[Question]
('Toulouse has twice as many sheep as Charleston. Charleston has 4 times as many sheep as Seattle. How many sheep do '
 'Toulouse, Charleston, and Seattle have together if Seattle has 20 sheep?')
[Answer] 260
[Predicted] 160

[Question]
('Carla is downloading a 200 GB file. Normally she can download 2 GB/minute, but 40% of the way through the download, '
 'Windows forces a restart to install updates, which takes 20 minutes. Then Carla has to restart the download from the '
 'beginning. How load does it take to download the file?')
[Answer] 160
[Predicted] 160



In [49]:
# zero-shot prompt w/ chain of thoughts
system = '''You are an excellent math solver to help user. 
The output format should be {{'chain_of_thought': 'step by step thinking process', 'final_answer': 'answer'}}.
Please answer the following question. 
'''

zero_shot_template = ChatPromptTemplate.from_messages(
    [
        ('system', system),
        ('human', 'Question: {question}'),
    ]
)

chain = (
    RunnablePassthrough()
    | model_ollama
    | output_parser
)

output = chain.batch(inputs=[
    zero_shot_template.format_messages(question=q['question']) for q in queries
])

print('[zero-shot prompt w CoT]')
for i, o in enumerate(output):
    print('[Question]')
    pprint(f'{queries[i]["question"]}', width=120)
    print(f'[Answer] {queries[i]["answer"]}')
    print(f'[Predicted] {o["final_answer"]}')
    print('[Predicted Chain of Thought]')
    for c in o["chain_of_thought"].split('\n'):
        print(c)
    print()

[zero-shot prompt w CoT]
[Question]
('Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only '
 '60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?')
[Answer] 64
[Predicted] $50
[Predicted Chain of Thought]
Here's how to solve the problem: 

* **Calculate the cost of the second and subsequent glasses:**  60% of $5 is (60/100) * $5 = $3.
* **Determine the number of full-priced and discounted glasses:** Kylar needs 16 glasses, so he'll buy 15 glasses at the discounted price and 1 at the full price.
* **Calculate the total cost of the discounted glasses:** 15 glasses * $3/glass = $45
* **Calculate the total cost:** $45 + $5 = $50

[Question]
('Toulouse has twice as many sheep as Charleston. Charleston has 4 times as many sheep as Seattle. How many sheep do '
 'Toulouse, Charleston, and Seattle have together if Seattle has 20 sheep?')
[Answer] 260
[Predicted] 260
[Predicted Chain of Thought]
1

In [50]:
# few-shot prompt w/ chain of thoughts
system = '''You are an excellent math solver to help user. 
The output format should be {{'chain_of_thought': 'step by step thinking process', 'final_answer': 'answer'}}.
Please answer the following question. 
'''

cot_example_template = ChatPromptTemplate.from_messages(
    [
        ('human', 'Question: {question}'),
        ('ai', '{{"chain_of_thought": "{cot}", "final_answer": "{answer}"}}'),
    ]
)

keys = ['question', 'cot', 'answer']
cot_few_shot_examples = FewShotChatMessagePromptTemplate(
    examples=[{k: e[k] for k in keys} for e in examples],   # filter with keys
    example_prompt=cot_example_template
)

cot_few_shot_template = ChatPromptTemplate.from_messages(
    [
        ('system', system),
        cot_few_shot_examples,
        ('human', 'Question: {question}'),
    ]
)

chain = (
    RunnablePassthrough()
    | model_ollama
    | output_parser
)

output = chain.batch(inputs=[
    cot_few_shot_template.format_messages(question=q['question']) for q in queries
])

print('[few-shot prompt w/ CoT]')
for i, o in enumerate(output):
    print('[Question]')
    pprint(f'{queries[i]["question"]}', width=120)
    print('[Chain of Thought]')
    for c in queries[i]["cot"].split('\n'):
        print(c)
    print(f'[Answer] {queries[i]["answer"]}')
    print(f'[Predicted] {o["final_answer"]}')
    print(f'[Predicted Chain of Thought]')
    for c in o["chain_of_thought"].split('\n'):
        print(c)
    print()

[few-shot prompt w/ CoT]
[Question]
('Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only '
 '60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?')
[Chain of Thought]
The discount price of one glass is 60/100 * 5 = $<<60/100*5=3>>3.
If every second glass is cheaper, that means Kylar is going to buy 16 / 2 = <<16/2=8>>8 cheaper glasses.
So for the cheaper glasses, Kylar is going to pay 8 * 3 = $<<8*3=24>>24.
And for the regular-priced glasses, Kylar will pay 8 * 5 = $<<8*5=40>>40.
So in total Kylar needs to pay 24 + 40 = $<<24+40=64>>64 for the glasses he wants to buy.
[Answer] 64
[Predicted] 64
[Predicted Chain of Thought]
Every other glass is discounted, so we have 16 / 2 = <<16/2=8>>8 full-priced glasses and 8 discounted glasses. 

The cost of the full-priced glasses is 8 * $5 = <<8*$5=$40>>$40.

The cost of the discounted glasses is 8 * ($5 * 0.60) = <<8*$5*0.60=$24>>$24.

The total c